In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB#Multinomial Naive Byes used for text classification or for discrete dataset while GaussianNB used for continuous data.
import nltk
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import re


In [6]:
!pip install nltk

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
df=pd.read_csv("IMDB Dataset.csv",encoding='latin1')

FileNotFoundError: [Errno 2] No such file or directory: 'IMDB Dataset.csv'

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(50000, 2)

In [ ]:
df['review'] = df['review'].str.lower()

In [ ]:
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

In [ ]:
def clean_text(text):
  words=re.sub('[^a-zA-Z]'," ",text).lower().split()
  words = [stemmer.stem(word) for word in words if word not in stop_words]
  cleaned_text = ' '.join(words)
  return cleaned_text

In [ ]:
df['clean_stem'] = df['review'].apply(clean_text)

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def lemmatize_text(text):
    words = re.sub('[^a-zA-Z]', ' ', text).lower().split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

In [ ]:
df['clean_Lamit']=df['review'].apply(lemmatize_text)

In [ ]:
df['y'] = pd.Categorical(df['sentiment']).codes

In [ ]:
df.head()

,review,sentiment,clean_stem,clean_Lamit,y
0,one of the other reviewers has mentioned that ...,positive,one review mention watch oz episod hook right ...,one reviewer mentioned watching oz episode hoo...,1
1,a wonderful little production. <br /><br />the...,positive,wonder littl product br br film techniqu unass...,wonderful little production br br filming tech...,1
2,i thought this was a wonderful way to spend ti...,positive,thought wonder way spend time hot summer weeke...,thought wonderful way spend time hot summer we...,1
3,basically there's a family where a little boy ...,negative,basic famili littl boy jake think zombi closet...,basically family little boy jake think zombie ...,0
4,"petter mattei's ""love in the time of money"" is...",positive,petter mattei love time money visual stun film...,petter mattei love time money visually stunnin...,1


In [ ]:
def tokenize_text(text):
    return word_tokenize(text)

In [ ]:
df['tokenized_clean_lamit'] = df['clean_Lamit'].apply(tokenize_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
x = cv.fit_transform(df['clean_Lamit']).toarray()

NameError: name 'df' is not defined

In [ ]:
y=df['y']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state = 0)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [ ]:
y_pred=spam_detect_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
score=accuracy_score(y_test,y_pred)
print(score)

0.8428666666666667


In [ ]:
# Creating the TFIDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=2500)
x = tv.fit_transform(df['clean_Lamit']).toarray()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [ ]:
y_pred=spam_detect_model.predict(X_test)

In [ ]:
score=accuracy_score(y_test,y_pred)
print(score)

0.8471


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      4910
           1       0.86      0.84      0.85      5090

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



#NOW WE WILL BUILT A MODEL WHICH WILL PREDICT THE NEXT WORD
## In first go we will fit it with the clean text clean text here means that text without stopwords

In [ ]:
text="""It is very seldom that mere ordinary people like John and myself secure
ancestral halls for the summer.
A colonial mansion, a hereditary estate, I would say a haunted house,and reach the height of romantic felicity--but that would be asking toomuch of fate!Still I will proudly declare that there is something queer about it.
Else, why should it be let so cheaply? And why have stood so long
untenanted?John laughs at me, of course, but one expects that in marriage.
John is practical in the extreme. He has no patience with faith, an
intense horror of superstition, and he scoffs openly at any talk of
things not to be felt and seen and put down in figures.
She didn't know I was in the room, and when I asked her in a quiet, a
very quiet voice, with the most restrained manner possible, what she
was doing with the paper--she turned around as if she had been caught
stealing, and looked quite angry--asked me why I should frighten her so!
Then she said that the paper stained everything it touched, that she had
found yellow smooches on all my clothes and John's, and she wished we
would be more careful!
Did not that sound innocent? But I know she was studying that pattern,
and I am determined that nobody shall find it out but myself!
Life is very much more exciting now than it used to be. You see I have
something more to expect, to look forward to, to watch. I really do eat
better, and am more quiet than I was.
John is so pleased to see me improve! He laughed a little the other day,
and said I seemed to be flourishing in spite of my wall-paper.
I turned it off with a laugh. I had no intention of telling him it was
BECAUSE of the wall-paper--he would make fun of me. He might even want
to take me away.
I don't want to leave now until I have found it out. There is a week
more, and I think that will be enough.
I'm feeling ever so much better! I don't sleep much at night, for it
is so interesting to watch developments; but I sleep a good deal in the
daytime.
In the daytime it is tiresome and perplexing.
There are always new shoots on the fungus, and new shades of yellow
all over it. I cannot keep count of them, though I have tried
conscientiously.
It is the strangest yellow, that wall-paper! It makes me think of all
the yellow things I ever saw--not beautiful ones like buttercups, but
old foul, bad yellow things.
But there is something else about that paper--the smell! I noticed it
the moment we came into the room, but with so much air and sun it was
not bad. Now we have had a week of fog and rain, and whether the windows
are open or not, the smell is here.
It creeps all over the house.
I find it hovering in the dining-room, skulking in the parlor, hiding in
the hall, lying in wait for me on the stairs.
It gets into my hair.
Even when I go to ride, if I turn my head suddenly and surprise
it--there is that smell!
Such a peculiar odor, too! I have spent hours in trying to analyze it,
to find what it smelled like.
It is not bad--at first, and very gentle, but quite the subtlest, most
enduring odor I ever met.
In this damp weather it is awful, I wake up in the night and find it
hanging over me.
It used to disturb me at first. I thought seriously of burning the
house--to reach the smell.
But now I am used to it. The only thing I can think of that it is like
is the COLOR of the paper! A yellow smell.
But I am here, and no person touches this paper but me--not ALIVE!
She tried to get me out of the room--it was too patent! But I said it
was so quiet and empty and clean now that I believed I would lie down
again and sleep all I could; and not to wake me even for dinner--I would
call when I woke.
So now she is gone, and the servants are gone, and the things are gone,
and there is nothing left but that great bedstead nailed down, with the
canvas mattress we found on it.
We shall sleep downstairs to-night, and take the boat home to-morrow.
I quite enjoy the room, now it is bare again.
How those children did tear about here!
This bedstead is fairly gnawed!
But I must get to work.
I have locked the door and thrown the key down into the front path.
I don't want to go out, and I don't want to have anybody come in, till
John comes.
I want to astonish him.
I've got a rope up here that even Jennie did not find. If that woman
does get out, and tries to get away, I can tie her!
But I forgot I could not reach far without anything to stand on!
This bed will NOT move!
I tried to lift and push it until I was lame, and then I got so angry I
bit off a little piece at one corner--but it hurt my teeth.
Then I peeled off all the paper I could reach standing on the floor.
It sticks horribly and the pattern just enjoys it! All those strangled
heads and bulbous eyes and waddling fungus growths just shriek with
derision!
I am getting angry enough to do something desperate. To jump out of the
window would be admirable exercise, but the bars are too strong even to
try.
Besides I wouldn't do it. Of course not. I know well enough that a step
like that is improper and might be misconstrued.
I don't like to LOOK out of the windows even--there are so many of those
creeping women, and they creep so fast.
I wonder if they all come out of that wall-paper as I did?
But I am securely fastened now by my well-hidden rope--you don't get ME
out in the road there!
And then I said it again, several times, very gently and slowly, and
said it so often that he had to go and see, and he got it of course, and
came in. He stopped short by the door.
"What is the matter?" he cried. "For God's sake, what are you doing!"
I kept on creeping just the same, but I looked at him over my shoulder.
"I've got out at last," said I, "in spite of you and Jane. And I've
pulled off most of the paper, so you can't put me back!"
Now why should that man have fainted? But he did, and right across my
path by the wall, so that I had to creep over him every time!
"""

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()

In [ ]:
print(text)

In [ ]:
text=text.lower()#Step 1 of text preprocessing to convert the whole text into lower case.

In [ ]:
print(text)

In [ ]:
tokenizer.fit_on_texts([text])#At this stage we have fit the tokenizer on the text without removing
                              #stop words or cleaning the text we will first fit model on raw text after that we will fit the model on the clean text


In [ ]:
len(tokenizer.word_index)#This line of code showing that we have total 424 no of unique words.

424

In [ ]:
tokenizer.word_index

{'i': 1,
 'the': 2,
 'and': 3,
 'it': 4,
 'to': 5,
 'of': 6,
 'that': 7,
 'is': 8,
 'but': 9,
 'in': 10,
 'a': 11,
 'so': 12,
 'me': 13,
 'not': 14,
 'paper': 15,
 'he': 16,
 'she': 17,
 'out': 18,
 'be': 19,
 'have': 20,
 'was': 21,
 'now': 22,
 'there': 23,
 'at': 24,
 'all': 25,
 'my': 26,
 'would': 27,
 'with': 28,
 'on': 29,
 'are': 30,
 'like': 31,
 'had': 32,
 'said': 33,
 'yellow': 34,
 'am': 35,
 'even': 36,
 "don't": 37,
 'very': 38,
 'john': 39,
 'for': 40,
 'room': 41,
 'we': 42,
 'more': 43,
 'did': 44,
 'find': 45,
 'you': 46,
 'wall': 47,
 'want': 48,
 'over': 49,
 'smell': 50,
 'get': 51,
 'reach': 52,
 'something': 53,
 'why': 54,
 'things': 55,
 'down': 56,
 'quiet': 57,
 'what': 58,
 'if': 59,
 'then': 60,
 'much': 61,
 'off': 62,
 'him': 63,
 'sleep': 64,
 'here': 65,
 'this': 66,
 'got': 67,
 'house': 68,
 'will': 69,
 'about': 70,
 'should': 71,
 'course': 72,
 'no': 73,
 'know': 74,
 'when': 75,
 'her': 76,
 'most': 77,
 'quite': 78,
 'angry': 79,
 'found': 80,
 

In [ ]:
#Here we are splitting our dataset into sentences on the basis of the line change
sentences=re.split('[.!]',text)#this line converting sentences ending with . and!
cleaned_sentences = [re.sub('["\,\-]', '', sentence).strip() for sentence in sentences if sentence]

for i in cleaned_sentences:
  print(i)

it is very seldom that mere ordinary people like john and myself secure
ancestral halls for the summer
a colonial mansion a hereditary estate i would say a haunted houseand reach the height of romantic felicitybut that would be asking toomuch of fate
still i will proudly declare that there is something queer about it
else why should it be let so cheaply? and why have stood so long
untenanted?john laughs at me of course but one expects that in marriage
john is practical in the extreme
he has no patience with faith an
intense horror of superstition and he scoffs openly at any talk of
things not to be felt and seen and put down in figures
she didn't know i was in the room and when i asked her in a quiet a
very quiet voice with the most restrained manner possible what she
was doing with the papershe turned around as if she had been caught
stealing and looked quite angryasked me why i should frighten her so
then she said that the paper stained everything it touched that she had
found yellow

In [ ]:
input_sequences = []
for sentence in text.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([text])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [ ]:
input_sequences

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
max_len = max([len(x) for x in input_sequences])

In [ ]:
max_len

1183

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [ ]:
padded_input_sequences

array([[  0,   0,   0, ...,   0,   4,   8],
       [  0,   0,   0, ...,   4,   8,  38],
       [  0,   0,   0, ...,   8,  38, 141],
       ...,
       [  0,   0,   4, ..., 140,  49,  63],
       [  0,   4,   8, ...,  49,  63, 423],
       [  4,   8,  38, ...,  63, 423, 424]], dtype=int32)

In [ ]:
X = padded_input_sequences[:,:-1]

In [ ]:
y = padded_input_sequences[:,-1]

In [ ]:
y.shape

(122928,)

In [ ]:
X.shape

(122928, 1182)

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=425)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
model = Sequential()
model.add(Embedding(425, 100, input_length=1182))
model.add(LSTM(150))
model.add(Dense(425, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1182, 100)         42500     
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 425)               64175     
                                                                 
Total params: 257275 (1004.98 KB)
Trainable params: 257275 (1004.98 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X,y,epochs=10)

Epoch 1/10
3842/3842 [==============================] - 181s 46ms/step - loss: 1.1281 - accuracy: 0.8060
Epoch 2/10
3842/3842 [==============================] - 174s 45ms/step - loss: 0.0075 - accuracy: 1.0000
Epoch 3/10
 346/3842 [=>............................] - ETA: 2:38 - loss: 0.0018 - accuracy: 1.0000